In [12]:
import pandas as pd

In [13]:
df=pd.read_csv('sentiment/spam_ham.txt',delimiter="\t",names=['target','msg'])

In [14]:

df.head()

,target,msg
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [15]:

df["word_count"]=df["msg"].apply(lambda x:len(str(x).split()))
df["char_count"]=df["msg"].apply(lambda x:len(x))
df.head()

,target,msg,word_count,char_count
0,ham,"Go until jurong point, crazy.. Available only ...",20,111
1,ham,Ok lar... Joking wif u oni...,6,29
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,28,155
3,ham,U dun say so early hor... U c already then say...,11,49
4,ham,"Nah I don't think he goes to usf, he lives aro...",13,61


In [16]:
from nltk.tokenize import RegexpTokenizer
rt=RegexpTokenizer('\w+')
def cleanText(s):
    w=rt.tokenize(s)
    return ' '.join(w)

In [17]:
rt=RegexpTokenizer('\w+')
w=rt.tokenize("Go until jurong point, crazy.. Available only ...")
w

['Go', 'until', 'jurong', 'point', 'crazy', 'Available', 'only']

In [18]:
' '.join(w)

'Go until jurong point crazy Available only'

In [19]:
df['msg']=df.msg.apply(cleanText)

In [20]:
df.head()

,target,msg,word_count,char_count
0,ham,Go until jurong point crazy Available only in ...,20,111
1,ham,Ok lar Joking wif u oni,6,29
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,28,155
3,ham,U dun say so early hor U c already then say,11,49
4,ham,Nah I don t think he goes to usf he lives arou...,13,61


In [21]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [22]:
tf=TfidfVectorizer(lowercase=True,stop_words='english')

In [23]:
from sklearn.model_selection import train_test_split

In [24]:
df.target=df.target.map({"ham":0,"spam":1})

In [25]:

x_tr,x_test,y_tr,y_test=train_test_split(df.msg,df.target,random_state=10)

In [26]:
x_tr_new=tf.fit_transform(x_tr).toarray()

In [27]:
len(tf.get_feature_names())

7179

In [28]:
from tensorflow.keras.layers import Dense,SimpleRNN
from tensorflow.keras.models import Sequential

In [29]:
df.head()

,target,msg,word_count,char_count
0,0,Go until jurong point crazy Available only in ...,20,111
1,0,Ok lar Joking wif u oni,6,29
2,1,Free entry in 2 a wkly comp to win FA Cup fina...,28,155
3,0,U dun say so early hor U c already then say,11,49
4,0,Nah I don t think he goes to usf he lives arou...,13,61


In [30]:
import numpy as np

In [31]:
a=x_tr_new.reshape((1,)+x_tr_new.shape)
a.shape

(1, 4179, 7179)

In [32]:
y_tr.shape

(4179,)

In [33]:
b=np.array(y_tr).reshape(-1,1)
b.shape

(4179, 1)

In [34]:
model=Sequential()
model.add(SimpleRNN(100,input_shape=(7179,),return_sequences=True))
model.compile(loss="binary_crossentropy",optimizer="adam",metrics=["accuracy"])
model.fit(np.array(a),np.array(b),batch_size=200,epochs=10)

ValueError: Input 0 of layer simple_rnn is incompatible with the layer: expected ndim=3, found ndim=2. Full shape received: [None, 7179]

In [35]:
model=Sequential()
model.add(Dense(100,input_dim=7179,activation="relu",kernel_initializer="he_normal",bias_initializer="he_normal"))
model.add(Dense(50,activation="relu",kernel_initializer="he_normal",bias_initializer="he_normal"))
model.add(Dense(25,activation="relu",kernel_initializer="he_normal",bias_initializer="he_normal"))
model.add(Dense(1,activation="sigmoid",kernel_initializer="he_normal",bias_initializer="he_normal"))
model.compile(loss="binary_crossentropy",optimizer="adam",metrics=["accuracy"])
h=model.fit(np.array(x_tr_new),np.array(y_tr),batch_size=200,epochs=10)

Train on 4179 samples
Epoch 1/10
4179/4179 [==============================] - 3s 701us/sample - loss: 0.8313 - accuracy: 0.2496
Epoch 2/10
4179/4179 [==============================] - 2s 444us/sample - loss: 0.4863 - accuracy: 0.8816
Epoch 3/10
4179/4179 [==============================] - 2s 471us/sample - loss: 0.2628 - accuracy: 0.8885
Epoch 4/10
4179/4179 [==============================] - 2s 462us/sample - loss: 0.1336 - accuracy: 0.9615
Epoch 5/10
4179/4179 [==============================] - 2s 461us/sample - loss: 0.0653 - accuracy: 0.9880
Epoch 6/10
4179/4179 [==============================] - 2s 495us/sample - loss: 0.0327 - accuracy: 0.9945
Epoch 7/10
4179/4179 [==============================] - 2s 488us/sample - loss: 0.0175 - accuracy: 0.9971
Epoch 8/10
4179/4179 [==============================] - 2s 465us/sample - loss: 0.0097 - accuracy: 0.9988
Epoch 9/10
4179/4179 [==============================] - 2s 474us/sample - loss: 0.0058 - accuracy: 0.9995 - loss: 0.0064 - accurac

In [36]:
model.evaluate(np.array(tf.transform(x_test).toarray()),np.array(y_test))

1393/1393 [==============================] - 1s 548us/sample - loss: 0.0786 - accuracy: 0.9734


[0.07855305721754662, 0.9734386]

In [37]:
from sklearn.externals import joblib
joblib.dump(tf,"tfidf.pkl")

['tfidf.pkl']

In [27]:
text1="you won cash 5000"
text2="hello shani how are you?"

In [28]:
t1=cleanText(text1)
t2=cleanText(text2)

In [29]:
test=np.array(tf.transform([t2]).toarray())

In [34]:
result=model.predict_classes(test)
s=pd.Series(result.flatten())
s=s.map({0:"ham",1:"spam"})
s[0]

'ham'

In [31]:
import matplotlib.pyplot as plt

In [1]:
from tensorflow.keras.models import load_model

In [2]:
model.save("email.h5")

NameError: name 'model' is not defined

In [3]:
model=load_model("email.h5")

In [4]:
from nltk.tokenize import RegexpTokenizer
rt=RegexpTokenizer('\w+')
def cleanText(s):
    w=rt.tokenize(s)
    return ' '.join(w)

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
tf=TfidfVectorizer(lowercase=True,stop_words='english')

In [6]:
text1="you won cash 5000 /"
text2="hello shani how are you?"

In [7]:
t1=cleanText(text1)
t2=cleanText(text2)

In [8]:
t1

'you won cash 5000'

In [9]:
tf.transform

<bound method TfidfVectorizer.transform of TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=None,
                min_df=1, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words='english', strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)>

In [40]:
a=tf.transform([t1,t2]).toarray()

NotFittedError: Vocabulary not fitted or provided

In [ ]:
import numpy as np

In [ ]:
test=np.array([t1,t2])
test

In [37]:
test=np.array(tf.transform([t2]).toarray())

NotFittedError: Vocabulary not fitted or provided

In [38]:
result=model.predict_classes(test)

In [6]:
model=load_model("email.h5")
a=joblib.load("model1.pkl")

In [5]:
from sklearn.externals import joblib

c:\users\hp\appdata\local\programs\python\python36\lib\site-packages\sklearn\externals\joblib\__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


In [8]:
from nltk.tokenize import RegexpTokenizer
rt=RegexpTokenizer('\w+')
def cleanText(s):
    w=rt.tokenize(s)
    return ' '.join(w)

In [11]:
import numpy as np
text1="you won 500000 cash"
t1=cleanText(text1)
test=np.array(a.transform([t1]).toarray())
result=model.predict_classes(test)
s=pd.Series(result.flatten())
pr=s.map({0:"Ham",1:"Spam"})
pr[0]

'Spam'